In [1]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import math
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters
from adam.geometry import linkParametric
from urdfpy import xyz_rpy_to_matrix, matrix_to_xyz_rpy

In [2]:
def SX2DM(sx):
    return cs.DM(sx)

# Loading the model 

In [3]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['l_lower_leg']
link_name_list_empty  = []
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
link_name = 'l_lower_leg'
link_characteristics = {link_name: r_hip_r_char}
joint_characteristics = {'l_ankle_pitch': r_hip_yaw_char}
print(joint_characteristics)
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link, link_characteristics, joint_characteristics)

kinDynNoParametric = KinDynComputations(urdf_path,joints_name_list, root_link)
kynDynParam_NoLink = KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list, link_name_list_empty, root_link)


{'l_ankle_pitch': <adam.geometry.linkParametric.JointCharacteristics object at 0x7f5c3874f940>}


Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor


### Variable definition

In [4]:
# Set joints and base informations
link_original = kinDyn.get_element_by_name(link_name, kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(0.058)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume

xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =density_num 
lenght_multiplier = 1
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

### Mass and Centroidal Matrix 

In [5]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()

# print("M", M(H_b, s, density, lenght_multiplier))
# print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

In [6]:
M_no_parametric = kinDynNoParametric.mass_matrix_fun()
M_parametric_no_link = kynDynParam_NoLink.mass_matrix_fun()
M_no_parametric_value = M_no_parametric(H_b, s)
# M_no_parametric_value = M_parametric_no_link(H_b,s,density,lenght_multiplier)
M_value = M(H_b, s, density, lenght_multiplier)
# print("M", M_no_parametric(H_b, s,density, lenght_multiplier))



In [7]:
error = 0 
for i in range(29):
    for j in range(29):
        err = float(M_no_parametric_value[i,j] - M_value[i,j])
        error += abs(err)
print('Total Error Mass Matrices', error)

Total Error Mass Matrices 1.2051873713413153e-12


In [8]:
J_cmm_no_parameric = kinDynNoParametric.centroidal_momentum_matrix_fun()
J_cmm_no_parameric_value = J_cmm_no_parameric(H_b,s)
J_cmm_parametric_no_link = kynDynParam_NoLink.centroidal_momentum_matrix_fun()
# J_cmm_no_parameric_value = J_cmm_parametric_no_link(H_b,s,density,lenght_multiplier)
J_cmm_value = J_cmm(H_b,s,density, lenght_multiplier)
error = 0 
for i in range(6):
    for j in range (29): 
        err = float(J_cmm_no_parameric_value[i,j] - J_cmm_value[i,j])
        error += abs(err)
print("Total error centroidal momentum matrix", error)



Total error centroidal momentum matrix 7.446574127987592e-13


### Total mass

In [9]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

mass:
 62.5739


In [10]:
mass_no_param = kinDynNoParametric.get_total_mass()
mass_param_no_link = kynDynParam_NoLink.get_total_mass()
# mass_no_param = mass_param_no_link(density,lenght_multiplier)
mass = kinDyn.get_total_mass()
mass_value = mass(density, lenght_multiplier)
error = float(mass_no_param-mass_value)
print("error mass", error)

error mass 0.0


### Forward Kinematics

In [11]:
H = kinDyn.forward_kinematics_fun('l_lower_leg')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

H left sole :
 @1=0, 
[[0.62561, 0.332075, 0.705931, 1.75912], 
 [0.492949, 0.533079, -0.687625, 2.23997], 
 [-0.60466, 0.778173, 0.169804, -1.77283], 
 [@1, @1, @1, 1]]


In [12]:

error_tot = 0 
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        H = kinDyn.forward_kinematics_fun(item)
        H_no_param = kinDynNoParametric.forward_kinematics_fun(item)
        H_link_no_param = kynDynParam_NoLink.forward_kinematics_fun(item)
        H_no_param_value = H_no_param(H_b, s)
        # H_no_param_value = H_link_no_param(H_b,s,density,lenght_multiplier)
        H_value = H(H_b, s, density,lenght_multiplier)
        error = 0 
        for i in range(4):
            for j in range(4):
                err = float(H_value[i,j]- H_no_param_value[i,j])
                error+= abs(err)
        error_tot += error
        # print("error forward kinematic",item, " ", error)
print("total error for the whole kin chain", error_tot)

total error for the whole kin chain 6.982470157623766e-13


### Center of Mass

In [13]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

CoM:
 [2.00615, 2.34927, -1.92541]


### Jacobian

In [14]:
CoM_no_param = kinDynNoParametric.CoM_position_fun()
CoM_param_no_link = kynDynParam_NoLink.CoM_position_fun()

CoM_no_param_value = CoM_no_param(H_b,s)
# CoM_no_param_value = CoM_param_no_link(H_b,s,density, lenght_multiplier)
CoM_value = com(H_b, s, density, lenght_multiplier)
error = 0
for i in range(3):
    err = float(CoM_no_param_value[i] - CoM_value[i])
    error+= abs(err)
print("error CoM", error)

error CoM 2.220446049250313e-16


In [15]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

Jacobian left sole
 @1=1, @2=0, 
[[@1, @2, @2, @2, 0.109618, -0.183376, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.128596, -0.199569, 0.0737475, 0.157854, 0.00229521, 0.0301411, @2, @2, @2, @2, @2, @2], 
 [@2, @1, @2, -0.109618, @2, -0.521129, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.14742, -0.491233, 0.102492, 0.211364, -0.0851748, 0.0349179, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @1, 0.183376, 0.521129, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.461043, 0.247582, 0.176853, -0.212155, 0.0573687, 0.0469243, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.19329, 0.350123, 0.936314, -0.332075, 0.332075, -0.231828, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.948889, 0.304259, -0.0864085, -0.533079, 0.533079, 0.845809, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.249497, 0.885912, -0.340366, -0.778173, 0.778173, -0.480483

In [16]:
error_tot = 0 
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        J_no_param = kinDynNoParametric.jacobian_fun(item)
        J_param_no_link = kynDynParam_NoLink.jacobian_fun(item)
        # J_no_param_value = J_param_no_link(H_b, s,density,lenght_multiplier)
        J_no_param_value = J_no_param(H_b, s)
        J_value = J(H_b,s, density, lenght_multiplier)
        error = 0 
        for i in range(6): 
            for j in range(29): 
                err = float(J_no_param_value[i,j]- J_value[i,j])
                error+ - abs(err)
        error_tot += error
        # print("error Jacobian", error)
print("error whole kin chain jacobian", error)

error whole kin chain jacobian 0


### Relative Jacobian

In [17]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))

Jacobian left sole relative:  
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.488752, 0.101007, 0.200608, -0.142124, 0.0330524, 0.0557178, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.489322, -0.0388744, -0.222982, 0.0955784, -0.0155998, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.109325, -0.304624, 0.0739283, 0.211382, -0.0179858, 0.0313339, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.949181, -0.309701, -0.904574, 0.904574, -0.2514, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0.178052, 0.247494, -0.247494, -0.967267, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.31473, 0.934014, -0.34712, 0.34712, -0.0345218, 0, 0, 0, 0, 0, 0]]


In [36]:
error_tot = 0
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        J_r_no_param = kinDynNoParametric.relative_jacobian_fun('l_sole')
        J_r_param_no_link = kynDynParam_NoLink.relative_jacobian_fun('l_sole')
        J_r_no_param_value = J_r_no_param(s)
        # J_r_no_param_value = J_r_param_no_link(s,density,lenght_multiplier)
        J_value = J_r(s, density, lenght_multiplier)
        error = 0 
        for i in range(6): 
            for j in range(23):
                err = float(J_value[i,j] - J_r_no_param_value[i,j])
                error += abs(err)
        error_tot += error
        # print("error relative jacobian", error)
print("error whole kin chain relative jacobian", error_tot)

error whole kin chain relative jacobian 0.0


In [37]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

tau: 
 [-29.0485, -39.1108, 591.87, 58.1481, 13.8899, -10.4976, 39.4674, 16.6687, 6.67221, 0.10654, 0.516076, 0.0764306, -1.11119, 4.28331, 1.98516, -0.0765226, -1.11005, 51.2712, 16.0727, 12.3119, -10.2667, 1.37228, 9.41598, 6.89902, -12.3027, 12.0212, -9.53551, 1.81583, -1.37282]


In [38]:
tau_no_parametric = kinDynNoParametric.rnea()
tau_parametric_no_link =kynDynParam_NoLink.rnea()
tau_no_parametric_value = tau_no_parametric(H_b, s, v_b, s_dot, gravity)
# tau_no_parametric_value = tau_parametric_no_link(H_b,s, v_b, s_dot, gravity,density,lenght_multiplier)
tau_value = tau(H_b, s, v_b, s_dot, gravity, density,  lenght_multiplier)

error = 0 
for i in range(29):
    err = float (tau_value[i] - tau_no_parametric_value[i])
    error += abs(err)

print("tau error", error)

tau error 2.1236346015029994e-12


In [39]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Bias force term:
 [-29.0485, -39.1108, 591.87, 58.1481, 13.8899, -10.4976, 39.4674, 16.6687, 6.67221, 0.10654, 0.516076, 0.0764306, -1.11119, 4.28331, 1.98516, -0.0765226, -1.11005, 51.2712, 16.0727, 12.3119, -10.2667, 1.37228, 9.41598, 6.89902, -12.3027, 12.0212, -9.53551, 1.81583, -1.37282]


In [40]:
h_no_param = kinDynNoParametric.bias_force_fun()
H_param_no_link= kynDynParam_NoLink.bias_force_fun()
h_no_param_values = H_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
# h_no_param_values = h_no_param(H_b,s,v_b,s_dot)
h_value = h(H_b,s,v_b,s_dot,density,lenght_multiplier)
error = 0
for i in range(29): 
    err = float(h_value[i]- h_no_param_values[i])
    error += abs(err)
print("bias forces error", error)

bias forces error 0.0


In [41]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Coriolis force term:
 [-29.0485, -39.1108, -21.7703, -5.91367, 17.9405, -10.4976, -4.60601, 2.82983, 3.73844, 1.65141, -0.264752, -0.0670193, -0.784092, 1.93812, -2.66984, 0.174215, -0.308823, 12.8441, -1.81793, 3.02014, -0.378731, 0.298746, 8.68097, 1.98228, -0.306164, 2.07712, -0.607938, 0.500264, -0.620515]


In [42]:
C_no_param = kinDynNoParametric.coriolis_term_fun()
C_param_no_link = kynDynParam_NoLink.coriolis_term_fun()
# C_no_param_value = C_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
C_no_param_value = C_no_param(H_b,s,v_b, s_dot)
C_value = C(H_b, s,v_b,s_dot,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(C_no_param_value[i] - C_value[i])
    error+= abs(err)
print("Coriolis error", error)

Coriolis error 2.3010343630502916e-12


In [43]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

Gravity term:
 [-2.13163e-14, -5.50671e-14, 613.64, 64.0618, -4.05066, -1.77636e-15, 44.0734, 13.8389, 2.93377, -1.54487, 0.780829, 0.14345, -0.327099, 2.34519, 4.65499, -0.250738, -0.801223, 38.4271, 17.8907, 9.29175, -9.88796, 1.07353, 0.735004, 4.91674, -11.9965, 9.94404, -8.92758, 1.31556, -0.752309]


In [44]:
G_no_param = kinDynNoParametric.gravity_term_fun()
G_param_no_link = kynDynParam_NoLink.gravity_term_fun()
G_no_param_value = G_no_param(H_b,s)
# G_no_param_value = G_param_no_link(H_b,s,density,lenght_multiplier)
G_value = G(H_b,s,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(G_no_param_value[i]-G_value[i])
    error += abs(err)
print("gravity term error", error)

gravity term error 2.317923630812402e-12


### Cylinder Link 

In [45]:
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
print(link_original.visuals[0].geometry.cylinder)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('length=', length)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])
link_parametric =linkParametric.linkParametric('l_lower_leg', 1.0,density_num, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
radius= 0.065
length= 0.2
origin= [-0.02       -0.015      -0.13        0.          0.         -1.57079633]
volume= 0.002654645792283376
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.02487067683333334
inertia yy= 0.02487067683333334
inertia zz= 0.011969087000000002
****************PARAMETRIC LINK****************
radius= 0.065
length= 0.2
origin= @1=0, [-0.02, -0.015, -0.13, @1, @1, -1.5708]
volume= 0.00265465
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.0248707
inertia yy= 0.0248707
inertia zz= 0.0119691


### Cylinder Link Parametric

In [46]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
radius= 0.065
length= (0.2*l_m)
origin= @1=0, [-0.02, -0.015, (-0.03-((0.2*l_m)/2)), @1, @1, -1.5708]
volume= (0.0132732*(0.2*l_m))
mass= ((0.0132732*(0.2*l_m))*rho)
density= rho
inertia xx= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia yy= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia zz= ((0.004225*((0.0132732*(0.2*l_m))*rho))/2)


### Box Link 

In [47]:
link_original = kinDyn.get_element_by_name('chest', kinDyn.robot)
chest_char = linkParametric.LinkCharacteristics(0.0, linkParametric.Side.DEPTH, False, False)

width = link_original.visuals[0].geometry.box.size[0]
depth = link_original.visuals[0].geometry.box.size[2]
height = link_original.visuals[0].geometry.box.size[1]

volume = width*depth*height
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('width=', width)
print('depth=', depth)
print('height=', height)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('chest', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.009188549344000001
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432180860748
inertia yy= 0.04098499815637501
inertia zz= 0.042206754510375005
****************PARAMETRIC LINK****************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.00918855
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432181
inertia yy= 0.040985
inertia zz= 0.0422068


### Box Link Parametric

In [48]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('chest', l_m,rho, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
width= 0.2041
depth= (0.2092*l_m)
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= (0.0439223*(0.2092*l_m))
mass= ((0.0439223*(0.2092*l_m))*rho)
density= rho
inertia xx= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.046311+sq(@1)))/12)
inertia yy= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.0416568+sq(@1)))/12)
inertia zz= ((0.0879679*((0.0439223*(0.2092*l_m))*rho))/12)


### Sphere Link

In [49]:
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)

radius = link_original.visuals[0].geometry.sphere.radius

volume = 4*(math.pi*radius**3)/3
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('l_elbow_1', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************ORIGINAL LINK******************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.01
inertia yy= 0.01
inertia zz= 0.01
****************PARAMETRIC LINK****************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.00014736256
inertia yy= 0.00014736256
inertia zz= 0.00014736256


### Sphere Link Parametric 

In [50]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_elbow_1', l_m,rho, kinDyn.robot, link_original)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************PARAMETRIC LINK****************
radius= (0.04*l_m)
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= @1=(0.04*l_m), ((12.5664*(@1*sq(@1)))/3)
mass= @1=(0.04*l_m), (((12.5664*(@1*sq(@1)))/3)*rho)
density= rho
inertia xx= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia yy= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia zz= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)


In [51]:
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
joint_original = kinDyn.get_joint_by_name('l_ankle_pitch',kinDyn.robot)
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
joint_parametric = linkParametric.jointParametric('l_ankle_pitch', link_parametric, joint_original, r_hip_yaw_char)
print(matrix_to_xyz_rpy(joint_original.origin))
print(joint_parametric.origin)

[ 0.       -0.0005   -0.255989  0.        0.        0.      ]
@1=0, [@1, -0.0005, (-0.055989-(0.2*l_m)), @1, @1, @1]


In [52]:
Sc = cs.SX.zeros(6)
Sc = np.random.rand(6,6)
print(Sc)
# print(cs.mtimes(Sc,Sc.T))
# print(Sc @ Sc.T)
print(Sc[3:,:3])

[[0.26013734 0.91572692 0.62387262 0.48513027 0.38359598 0.23940381]
 [0.09161378 0.55897348 0.07992375 0.18575859 0.62381135 0.88294485]
 [0.31941291 0.76208279 0.94695524 0.41804689 0.97320584 0.53197286]
 [0.84225329 0.1748389  0.04545098 0.3641277  0.17383929 0.93780122]
 [0.04013763 0.68949543 0.68371738 0.71946945 0.69479072 0.32295439]
 [0.72990134 0.60026471 0.91899363 0.79274674 0.34982961 0.78831506]]
[[0.84225329 0.1748389  0.04545098]
 [0.04013763 0.68949543 0.68371738]
 [0.72990134 0.60026471 0.91899363]]


In [53]:
I = linkParametric.I_parametric(); 
I.ixx = 1
I.iyy = 2
I.izz = 3

I_float = np.zeros([3,3])
I_float[0,0] = 1 
I_float[1,1] = 2 
I_float[2,2] = 3

a = utils.spatial_inertial_with_parameter(I,1,[0,0,0], [0,0,0])
b = utils.spatial_inertia(I_float, 1,[0,0,0], [0,0,0])

print(a)
print(b)

@1=1, @2=0, 
[[@1, 00, 00, @2, @2, @2], 
 [00, @1, 00, @2, @2, @2], 
 [00, 00, @1, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2], 
 [@2, @2, @2, @2, 2, @2], 
 [@2, @2, @2, @2, @2, 3]]
[[ 1.  0.  0.  0.  0. -0.]
 [ 0.  1.  0. -0.  0.  0.]
 [ 0.  0.  1.  0. -0.  0.]
 [ 0. -0.  0.  1.  0.  0.]
 [ 0.  0. -0.  0.  2.  0.]
 [-0.  0.  0.  0.  0.  3.]]
